# Optimization Problem: Maximize Profit for smartphone sales

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pulp import *
import datapane as dp

In [2]:
#read original dataset
ecommerce_data = pd.read_csv('kz.csv', parse_dates=['event_time'])
ecommerce_data.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39+00:00,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39+00:00,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43+00:00,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43+00:00,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21+00:00,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [3]:
#transform and clean data
def transform_dataset(data):
    """
    Get a clean subset of apple and samsung smartphones in 2020 that includes the focus columns with no missing values or data quality issue
    """
    focus_fields = data.loc[:, ['event_time', 'category_code', 'brand', 'price']]
    focus_rows = focus_fields.dropna(subset=['price', 'category_code'])
    focus_rows = focus_rows[focus_rows['event_time'].dt.year ==2020]
    focus_rows['year'] = focus_rows['event_time'].dt.year
    focus_rows['category'], focus_rows['item'] = focus_rows['category_code'].str.split('.').str[0], focus_rows['category_code'].str.split('.').str[-1]
    focus_rows['date'] = focus_rows['event_time'].dt.date
    focus_data = focus_rows[(focus_rows['item']=='smartphone') & (focus_rows['brand'].isin(['samsung','apple']))]
    smartphone_data = focus_data.loc[:,['date', 'event_time', 'brand', 'price']]
    return smartphone_data


In [4]:
# return clean subset data
smartphone_data = transform_dataset(ecommerce_data)
smartphone_data.head()

,date,event_time,brand,price
6,2020-04-26,2020-04-26 09:33:47+00:00,apple,1387.01
7,2020-04-26,2020-04-26 09:33:47+00:00,apple,1387.01
8,2020-04-26,2020-04-26 09:33:47+00:00,apple,1387.01
9,2020-04-26,2020-04-26 09:33:47+00:00,apple,1387.01
27,2020-04-29,2020-04-29 04:46:39+00:00,samsung,300.90


In [5]:
# expose the datafram to datapane to share the table publicly
r = dp.Report(dp.DataTable(smartphone_data.head(20)))
r.publish(name='you', visibility='PUBLIC')

/root/venv/lib/python3.7/site-packages/datapane/client/api/report/core.py:510: UserWarning: use dp.ReportFormatting instead when calling upload or save
  warnings.warn("use dp.ReportFormatting instead when calling upload or save")
/root/venv/lib/python3.7/site-packages/datapane/client/api/report/core.py:577: UserWarning: Deprecated - please use report.upload instead
  warnings.warn("Deprecated - please use report.upload instead")
/root/venv/lib/python3.7/site-packages/datapane/client/api/report/core.py:557: UserWarning: Visibility parameter deprecated, your reports are drafts by default and can be published via the report share feature in your browser
  "Visibility parameter deprecated, your reports are drafts by default and can be published via the report share feature in your browser"


Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/anita8/reports/you/) to view your report and optionally share it with the Datapane Community

### Analysis the data
- The fixed price we want to use
- The capacity constraint(s)
- The revenue we make currently in a day on an average

In [6]:

def average_price(data):
    """
    Get the average price per brand by summing the unique variation of prices over time and dividing it by the count of variations
    """
    brand_price = {}
    for i in list(data['brand'].unique()):
        list_price_sum = sum(data[data['brand']==i]['price'].unique())
        unique_price_count = data[data['brand']==i]['price'].nunique()
        avg_price = list_price_sum/unique_price_count
        brand_price[i] = round(avg_price , 2)
    return(brand_price)
        
average_price_per_brand = average_price(smartphone_data)

In [7]:

def average_phone_available(data):
    """
    Get the average daily stock of phones by getting all the phones sold so far and dividing it by the number of days recorded
    """
    inventory_sold = data.shape[0]
    days_so_far = data['date'].nunique()
    average_inventory_available = inventory_sold/days_so_far
    return(round(average_inventory_available,0))

average_inventory_available = average_phone_available(smartphone_data)

In [8]:
def average_revenue(data):
    """
    Get the average revenue made in a day by getting the sum of revenue made so far divided by the number of days recorded
    """
    revenue_so_far = sum(data['price'])
    days_so_far = data['date'].nunique()
    average_daily_revenue = revenue_so_far/days_so_far
    return (round(average_daily_revenue,2))

revenue_daily_average = average_revenue(smartphone_data)



### Use Pulp to solve the optimization problem

In [9]:
# solve the optimization problem

model = LpProblem("Maximize sales revenue", LpMaximize)
A = LpVariable('Apple', lowBound=0, cat='Integer')
S = LpVariable('Samsung', lowBound=0, cat='Integer')
model += average_price_per_brand['apple'] * A + average_price_per_brand['samsung'] * S
model += 1* A + 1 * S <= average_inventory_available
status = model.solve()
print("Stock {} Apple Phones ".format(A.varValue))
print("Stock {} Samsung Phones".format(S.varValue))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /root/venv/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/25c85825af6d4a989318a72dd567ed66-pulp.mps max branch printingOptions all solution /tmp/25c85825af6d4a989318a72dd567ed66-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 15 RHS
At line 17 BOUNDS
At line 20 ENDATA
Problem MODEL has 1 rows, 2 columns and 2 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 547240 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -547240 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seco

### Make Decision

In [10]:
# get the maximum revenue with the option A & S stock quantity
print(average_price_per_brand['apple'] * A.varValue + average_price_per_brand['samsung'] * S.varValue)

547239.82


In [11]:
# Compare the optimum solution revenue with the current average revenue made
# get the percentage increase/decrease
maximum_revenue = average_price_per_brand['apple'] * A.varValue + average_price_per_brand['samsung'] * S.varValue
current_revenue = average_revenue(smartphone_data)

print(str(round(((maximum_revenue - current_revenue)/ current_revenue) * 100.0,2)) + '%')

120.69%


It is better to stick to a fixed price through out the day

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8606e20e-0dce-4234-a218-aa8a4a0a031e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>